In [21]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from scipy.stats import norm
from scipy.interpolate import griddata
from datetime import datetime




In [ ]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    #
    """
    Calculates the theoretical price of an option using Black-Scholes formula
    S = Spot Price
    K = Strik Price
    T = Time in years
    r = Risk-free rate
    sigma = Volatility
    """
    if T <= 0 or sigma <= 0:
        return 0.0

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

def black_scholes_vega(S, K, T, r, sigma):
    """
    Calculates Vega
    """
    if T <= 0 or sigma <= 0:
        return 0.0
        
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * norm.pdf(d1)
    return vega

def newton_raphson_iv(market_price, S, K, T, r, option_type='call', tol=1e-5, max_iter=100):
    """
    Solver that finds implied volatility matching market price
    """
    sigma = 0.5 # Initial guess
    
    for i in range(max_iter):
        price = black_scholes_price(S, K, T, r, sigma, option_type)
        vega = black_scholes_vega(S, K, T, r, sigma)
        
        diff = market_price - price
        
        if abs(diff) < tol:
            return sigma
        
        # Avoid division by zero for deep ITM/OTM options
        if abs(vega) < 1e-8:
            return np.nan 
            
        sigma = sigma + diff / vega
        
        if sigma <= 0 or sigma > 5.0:
            sigma = 0.5
            
    return np.nan

In [ ]:
def fetch_spy_data(ticker_symbol="SPY", risk_free_rate=0.045):
    print(f"Fetching data for {ticker_symbol}...")
    spy = yf.Ticker(ticker_symbol)
    
    try:
        current_price = spy.history(period="1d")['Close'].iloc[-1]
    except Exception:
        print("Error fetching spot price.")
        return pd.DataFrame()

    print(f"Current Spot Price: ${current_price:.2f}")
    
    expirations = spy.options
    all_options = []
    
    for date in expirations[:12]: 
        try:
            chain = spy.option_chain(date)
            calls = chain.calls
            
            exp_date = datetime.strptime(date, "%Y-%m-%d")
            today = datetime.now()
            days_to_expiry = (exp_date - today).days
            T = days_to_expiry / 365.0
            
            if T < (2/365.0):
                continue

            for index, row in calls.iterrows():
                if row['volume'] == 0 or np.isnan(row['volume']):
                    continue
                
                bid = row['bid']
                ask = row['ask']
                if bid == 0 or ask == 0:
                    continue
                
                # If the difference between Ask and Bid is too wide its junk data
                spread = ask - bid
                midpoint_price = (bid + ask) / 2
                
                # If spread is > 40% of the price skip it
                if (spread / midpoint_price) > 0.4:
                    continue

                strike = row['strike']
                
                iv = newton_raphson_iv(midpoint_price, current_price, strike, T, risk_free_rate, 'call')
                
                if not np.isnan(iv):
                    all_options.append({
                        'Strike': strike,
                        'Time': T,
                        'Volatility': iv,
                        'Price': midpoint_price
                    })
        except Exception:
            continue
            
    print(f"Processed {len(all_options)} valid options contracts.")
    return pd.DataFrame(all_options)

print("Data pipeline updated with Spread Filter.")

Data pipeline updated with Spread Filter.


In [ ]:
# Fetch the data
df = fetch_spy_data()

# Check if we have data
if not df.empty:
    # Create the grid
    x = df['Strike']
    y = df['Time']
    z = df['Volatility']

    xi = np.linspace(x.min(), x.max(), 50)
    yi = np.linspace(y.min(), y.max(), 50)
    X, Y = np.meshgrid(xi, yi)

    # Interpolate
    Z = griddata((x, y), z, (X, Y), method='linear')

    # Plot
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Viridis')])

    fig.update_layout(
        title='SPY Implied Volatility Surface (Call Options)',
        scene = dict(
            xaxis_title='Strike Price ($)',
            yaxis_title='Time to Maturity (Years)',
            zaxis_title='Implied Volatility',
        ),
        width=900,
        height=700,
    )

    fig.show()
else:
    print("No data found")

Fetching data for SPY...
Current Spot Price: $691.97
Processed 683 valid options contracts.
